<a href="https://colab.research.google.com/github/LonnieSly/grocery_inventory_automation_ml/blob/main/grocery_inventory_automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

df = pd.read_csv("grocery_data.csv")

df.head()


,Product_Name,Catagory,Supplier_Name,Warehouse_Location,Status,Product_ID,Supplier_ID,Date_Received,Last_Order_Date,Expiration_Date,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Sales_Volume,Inventory_Turnover_Rate,percentage
0,Bell Pepper,Fruits & Vegetables,Eimbee,20 Pennsylvania Parkway,Discontinued,29-017-6255,43-348-2450,3/1/2024,1/6/2025,1/31/2025,46,64,17,$4.60,96,55,1.96%
1,Vegetable Oil,Oils & Fats,Digitube,03643 Oakridge Lane,Backordered,79-569-8856,04-854-7165,4/1/2024,5/19/2024,6/11/2024,51,87,86,$2.00,24,83,0.91%
2,Parmesan Cheese,Dairy,BlogXS,73 Graedel Street,Discontinued,28-146-2641,82-995-0739,4/1/2024,12/21/2024,4/8/2024,38,67,66,$12.00,35,24,1.36%
3,Carrot,Fruits & Vegetables,Avaveo,44801 Myrtle Center,Discontinued,11-581-9869,22-867-3079,5/1/2024,12/12/2024,9/26/2024,51,60,98,$1.50,44,95,1.36%
4,Garlic,Fruits & Vegetables,Katz,6195 Monterey Center,Discontinued,13-202-4809,24-281-7685,5/1/2024,7/28/2024,5/20/2024,27,22,89,$7.00,91,77,2.17%


In [ ]:
# Convert date columns to datetime
df['Date_Received'] = pd.to_datetime(df['Date_Received'], errors='coerce')
df['Expiration_Date'] = pd.to_datetime(df['Expiration_Date'], errors='coerce')
df['Last_Order_Date'] = pd.to_datetime(df['Last_Order_Date'], errors='coerce')

# Preview to make sure it worked
df[['Product_Name', 'Date_Received', 'Expiration_Date']].head()


,Product_Name,Date_Received,Expiration_Date
0,Bell Pepper,2024-03-01,2025-01-31
1,Vegetable Oil,2024-04-01,2024-06-11
2,Parmesan Cheese,2024-04-01,2024-04-08
3,Carrot,2024-05-01,2024-09-26
4,Garlic,2024-05-01,2024-05-20


In [ ]:
# 🔻 LOW-STOCK ALERTS: Stock is below or equal to reorder level
low_stock = df[df['Stock_Quantity'] <= df['Reorder_Level']]

# ❌ EXPIRED ITEMS: Items past their expiration date
expired_items = df[df['Expiration_Date'] < pd.Timestamp.today()]

# 📈 TOP-SELLING PRODUCTS: Sort by sales volume
top_sellers = df.sort_values(by='Sales_Volume', ascending=False).head(10)

# 👀 Preview each
print("LOW STOCK PRODUCTS:")
print(low_stock[['Product_Name', 'Stock_Quantity', 'Reorder_Level']].head())

print("\nEXPIRED ITEMS:")
print(expired_items[['Product_Name', 'Expiration_Date']].head())

print("\nTOP SELLERS:")
print(top_sellers[['Product_Name', 'Sales_Volume']].head())


LOW STOCK PRODUCTS:
      Product_Name  Stock_Quantity  Reorder_Level
0      Bell Pepper              46             64
1    Vegetable Oil              51             87
2  Parmesan Cheese              38             67
3           Carrot              51             60
6    Coconut Sugar              17             85

EXPIRED ITEMS:
      Product_Name Expiration_Date
0      Bell Pepper      2025-01-31
1    Vegetable Oil      2024-06-11
2  Parmesan Cheese      2024-04-08
3           Carrot      2024-09-26
4           Garlic      2024-05-20

TOP SELLERS:
        Product_Name  Sales_Volume
452      White Sugar           100
557  Long Grain Rice           100
589    Egg (Chicken)           100
916  Parmesan Cheese           100
576             Lime           100


In [ ]:
# Save all filtered data as CSV files
low_stock.to_csv("low_stock_report.csv", index=False)
expired_items.to_csv("expired_items_report.csv", index=False)
top_sellers.to_csv("top_sellers_report.csv", index=False)

print("✅ Reports saved!")


✅ Reports saved!


In [ ]:
## Grocery Inventory Automation – Phase 2
## Predicting Sales Volume Using Linear Regression


In [ ]:
# Remove dollar signs and convert Unit_Price to float
df['Unit_Price'] = df['Unit_Price'].replace('[\$,]', '', regex=True).astype(float)


In [ ]:
# Select correct columns for features and target
features = df[['Unit_Price', 'Stock_Quantity', 'Reorder_Level', 'Catagory', 'Supplier_Name']]
target = df['Sales_Volume']


In [ ]:
# Categorical and numerical columns
categorical_cols = ['Catagory', 'Supplier_Name']
numerical_cols = ['Unit_Price', 'Stock_Quantity', 'Reorder_Level']

# One-hot encode text features and pass through numeric ones
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  # keep numerical columns as-is
)

# Split data: 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

In [ ]:
# Create a pipeline: preprocessor + model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train the model on the training data
model.fit(X_train, y_train)

print("✅ Model trained successfully!")

✅ Model trained successfully!


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate performance
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"📈 R² Score: {r2:.2f}")
print(f"📉 Mean Absolute Error: {mae:.2f} units")


📈 R² Score: -0.46
📉 Mean Absolute Error: 20.89 units
